Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.2813,  0.3192, -0.2885, -0.1653,  0.0576,  0.2308],
        [-0.2357,  0.0409,  0.0449,  0.3399, -0.1957,  0.0268],
        [-0.2226,  0.3821,  0.2827,  0.3517,  0.3630, -0.1983],
        [ 0.2283, -0.1590, -0.1372,  0.4071, -0.3634, -0.1149],
        [ 0.0108, -0.3122, -0.2177,  0.1462, -0.3974,  0.3717],
        [ 0.1158, -0.2667,  0.3847,  0.3460, -0.1004, -0.3446],
        [ 0.2710,  0.0979, -0.1606, -0.2049, -0.3926, -0.2159],
        [-0.3774,  0.0109, -0.2323, -0.3531,  0.3913, -0.2346],
        [ 0.3710, -0.3454, -0.2748, -0.2481, -0.0429,  0.1095],
        [-0.1920, -0.1252,  0.2779, -0.3012, -0.0562, -0.2491],
        [ 0.1947,  0.0914,  0.2640, -0.3526, -0.0520,  0.1303],
        [-0.2898,  0.0030, -0.3551,  0.1938,  0.2293, -0.3794],
        [-0.3314,  0.0477,  0.1893,  0.2684,  0.0328, -0.0306],
        [ 0.3554,  0.1822, -0.0543,  0.3923, -0.1457,  0.1295],
        [ 0.3501,  0.0962, -0.0181,  0.1966,  0.0066, -0.2147],
        [-0.1229, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.1013],
        [0.0977],
        [0.0974],
        ...,
        [0.1029],
        [0.0951],
        [0.0960]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0524)
1 tensor(0.0500)
2 tensor(0.0477)
3 tensor(0.0458)
4 tensor(0.0442)
5 tensor(0.0429)
6 tensor(0.0419)
7 tensor(0.0412)
8 tensor(0.0407)
9 tensor(0.0405)
10 tensor(0.0405)
11 tensor(0.0405)
12 tensor(0.0404)
13 tensor(0.0403)
14 tensor(0.0401)
15 tensor(0.0399)
16 tensor(0.0395)
17 tensor(0.0391)
18 tensor(0.0387)
19 tensor(0.0382)
20 tensor(0.0378)
21 tensor(0.0373)
22 tensor(0.0368)
23 tensor(0.0364)
24 tensor(0.0360)
25 tensor(0.0356)
26 tensor(0.0352)
27 tensor(0.0349)
28 tensor(0.0346)
29 tensor(0.0342)
30 tensor(0.0339)
31 tensor(0.0335)
32 tensor(0.0332)
33 tensor(0.0328)
34 tensor(0.0324)
35 tensor(0.0320)
36 tensor(0.0315)
37 tensor(0.0311)
38 tensor(0.0306)
39 tensor(0.0301)
40 tensor(0.0297)
41 tensor(0.0292)
42 tensor(0.0287)
43 tensor(0.0282)
44 tensor(0.0277)
45 tensor(0.0271)
46 tensor(0.0266)
47 tensor(0.0259)
48 tensor(0.0252)
49 tensor(0.0244)
50 tensor(0.0236)
51 tensor(0.0228)
52 tensor(0.0221)
53 tensor(0.0214)
54 tensor(0.0207)
55 tensor(0.0200)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1465.67903376]
[1409.99999118] [1555.84601068]
[1128.99997044] [1151.88402462]
[12991.99967575] [11840.74460983]
[14720.0001564] [14620.01478195]
[2938.99994564] [2651.99620247]
[6635.00014687] [8077.39010239]
[11215.99973297] [9239.78925896]
[1040.99999189] [1388.57040977]
[5040.00003624] [5209.49307251]
[9541.99988747] [8453.9090004]
[3792.99988461] [5327.86727142]
[1002.0000155] [754.84396195]
[2542.00005341] [1513.48542976]
[6788.00001717] [6589.84114075]
[9386.00025749] [8229.77701187]
[1245.99996853] [1701.33504391]
[997.00000262] [762.51327586]
[708.00000262] [697.36696529]
[1435.99997544] [1451.01834226]
[2590.00006676] [2447.16283321]
[1392.99997497] [1670.33876753]
[11755.99946976] [9411.23913956]
[2007.99997377] [1323.42058516]
[2707.99998474] [2568.90225315]
[631.00000417] [577.83354282]
[652.99999881] [975.46210575]
[2492.00006247] [2673.75162601]
[594.99999416] [548.20313001]
[760.0000056] [1097.02995729]
[1179.00003028] [1100.66679382]
[1789.9999361] [17

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [800.03259635]
[628.00000334] [729.71856308]
[442.00000328] [718.30313253]
[2284.99997044] [2637.35211658]
[1040.99999189] [799.81896269]
[809.99999654] [1266.74809957]
[3049.99987316] [3498.14395905]
[720.00000596] [588.52280498]
[877.99999249] [676.94522762]
[1292.00000429] [1623.77290058]
[7056.00004578] [4888.91318798]
[4176.99999142] [5934.66154671]
[13701.00020599] [15297.66280746]
[2948.0000515] [4154.83252811]
[16732.99985504] [14280.25368881]
[1815.00000048] [2115.74864578]
[16220.00016022] [12154.84072495]
[5967.00019073] [4579.58901691]
[473.00000042] [670.9624517]
[1078.9999795] [1105.37121296]
[452.00000319] [555.49981666]
[801.99998283] [803.99176097]
[2458.99994993] [2537.55691433]
[2492.99998236] [2503.61972141]
[582.00000203] [669.66797161]
[1405.9999671] [1437.16092157]
[8715.99991035] [11688.03358078]
[7634.00004578] [6492.88749695]
[506.99999839] [557.81936622]
[665.99999094] [730.04578328]
[1815.00000048] [1610.03337288]
[707.00001383] [618.56587768]

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1379.2987298257003


## Conclusão



Após conhecer em detalhes cada etapa e processo das redes neurais, a partir da programação de uma rede neural nos notebooks anteriores, neste notebook, foi realizado todo o processo de montagem e treino de uma rede neural a partir da biblioteca pytorch, o que possibilitou uma abordagem mais enxuta e eficiente, ou seja, uma abordagem aplicável para a resolução de um problema real. Por fim, foi possível aplicar e entender todos os processos utilizados anteriormente, agora, com a biblioteca pytorch, possibilitando a criação de um modelo preditivo razoável.

## Playground

